# FABRIC Reference Experiment

This notebook has the instructions to run a reference (baseline) experiment on FABRIC testbed to set up the multipath topology below
and then configure the FABRIC nodes to run bulk data transfer between the client and server nodes over one Multipath TCP connection.

![cloudLabtopology](cloudlabExperimentTopology.png)

* Set up the Environment & reserve and configure resources on FABRIC
* validate the multipath configuration
* "play back" link traces on the multipath configuration



## Reserve and configure resources on FABRIC

First, set up your enrivonment in FABRIC.

It is assumed that the keys, the FABRIC config file, and the SSH config file are already created (in the "Configure your environment" notebook) using those default settings (paths, etc).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

In [ ]:
slice_name="mptcp-demo"

### Use previously reserved resources (if needed)

If this raises an error, you don't have a slice for this experiment and need to create one (next section):

In [ ]:
if fablib.get_slice(slice_name):
    print("You already have a slice named %s - skip the next part." % slice_name)
    slice = fablib.get_slice(name=slice_name)
    print(slice)

### Create slice and reserve resources (only if you don't already have slice)

See available resources:

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Create a slice:

In [ ]:
# Create a slice
slice = fablib.new_slice(name=slice_name)

Pick a random site and show it:

In [ ]:
site_name = fablib.get_random_site()
site_name

Add nodes to the slice (note: use Ubuntu 18 because the MPTCP kernel is based on kernel 4.19):

In [ ]:
n_c  = slice.add_node(name="client",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_s  = slice.add_node(name="server",    site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r1 = slice.add_node(name="router1",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e1 = slice.add_node(name="emulator1", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_r2 = slice.add_node(name="router2",   site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')
n_e2 = slice.add_node(name="emulator2", site=site_name, cores=4, ram=32, disk=100, image='default_ubuntu_18')

Add network interfaces to the nodes:

In [ ]:
if_c1     =  n_c.add_component(model="NIC_Basic", name="if_c1").get_interfaces()[0]
if_c2     =  n_c.add_component(model="NIC_Basic", name="if_c2").get_interfaces()[0]
if_s1     =  n_s.add_component(model="NIC_Basic", name="if_s1").get_interfaces()[0]
if_s2     =  n_s.add_component(model="NIC_Basic", name="if_s2").get_interfaces()[0]
if_e1_l   = n_e1.add_component(model="NIC_Basic", name="if_e1_l").get_interfaces()[0]
if_e1_r   = n_e1.add_component(model="NIC_Basic", name="if_e1_r").get_interfaces()[0]
if_r1_l   = n_r1.add_component(model="NIC_Basic", name="if_r1_l").get_interfaces()[0]
if_r1_r   = n_r1.add_component(model="NIC_Basic", name="if_r1_r").get_interfaces()[0]
if_e2_l   = n_e2.add_component(model="NIC_Basic", name="if_e2_l").get_interfaces()[0]
if_e2_r   = n_e2.add_component(model="NIC_Basic", name="if_e2_r").get_interfaces()[0]
if_r2_l   = n_r2.add_component(model="NIC_Basic", name="if_r2_l").get_interfaces()[0]
if_r2_r   = n_r2.add_component(model="NIC_Basic", name="if_r2_r").get_interfaces()[0]

and show them:

In [ ]:
print(f"{slice.list_interfaces()}")

add Ethernet networks to the slice:

In [ ]:
net_1_c_e = slice.add_l2network(name='net_1_c_e', type='L2Bridge', interfaces=[if_c1,    if_e1_l])
net_2_c_e = slice.add_l2network(name='net_2_c_e', type='L2Bridge', interfaces=[if_c2,    if_e2_l])
net_1_e_r = slice.add_l2network(name='net_1_e_r', type='L2Bridge', interfaces=[if_e1_r,  if_r1_l])
net_2_e_r = slice.add_l2network(name='net_2_e_r', type='L2Bridge', interfaces=[if_e2_r,  if_r2_l])
net_1_r_s = slice.add_l2network(name='net_1_r_s', type='L2Bridge', interfaces=[if_r1_r,  if_s1])
net_2_r_s = slice.add_l2network(name='net_2_r_s', type='L2Bridge', interfaces=[if_r2_r,  if_s2])

and see them:

In [ ]:
print(f"{slice.list_interfaces()}")

Note: Basic NICs claim 0 bandwidth but are 100 Gbps shared by all Basic
NICs on the host.

Submit slice (reserve resources):

Now we are ready to reserve the resources and get the login details.

This step will take a little while, as we communicate with FABRIC to
reserve our requested resources.

-   As it runs, you will see the “State” of each node go from “Ticketed”
    to “Active” (and a “Management IP”, which may be IPv4 or IPv6, will
    be assigned to each one)
-   Then, you’ll wait a few more minutes for some `wait_ssh` and
    `post_boot_config` processes to run.
-   Next, it will show the MAC address and interface name of each of the
    “dataplane” interfaces you set up, as these come up

In [ ]:
slice.submit()

Our final slice status should be “StableOK”:

In [ ]:
print(f"{slice}")

we can block until all hosts are ready to login:

In [ ]:
slice.wait_ssh(progress=True)

Test that resources are available over SSH - should return True:

In [ ]:
slice.test_ssh()

Note: now that the slice is reserved, the variables from the previous section (node and interface variables) are no longer useful. 

### How to Login to Resources in FABRIC?

Run the following cell, and observe the table output - you will see an SSH command for each of the nodes in the topology topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, if needed, you can open an SSH session on any of the nodes as follows: 
- in Jupyter, from the menu bar, use File > New > Terminal to open a new terminal.
- copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

### How to Renew your slice lease in FABRIC (if you already have a slice)?

In [ ]:
import datetime
from datetime import timedelta

now = datetime.datetime.now(datetime.timezone.utc)
end_date = (now + timedelta(days=10)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [ ]:
slice.show()

If you don't see the slice lease renewed as a result of the above slice information, you can login to FABRIC web portal and check slice lease time there.

### Install MPTCP kernel on client and server

First, we download the packages to the Jupyter workspace (the nodes cannot retrieve them directly from Github because of [the IPv6 issue](https://learn.fabric-testbed.net/knowledge-base/using-ipv4-only-resources-like-github-or-docker-hub-from-ipv6-fabric-sites/).) 

In [ ]:
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb -O /tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-libc-dev_20220311125841-1_amd64.deb -O /tmp/linux-libc-dev_20220311125841-1_amd64.deb
!wget https://github.com/multipath-tcp/mptcp/releases/download/v0.95.2/linux-mptcp_v0.95.2_20220311125841-1_all.deb -O /tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb

Then we transfer over to the hosts and install.

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_file('/tmp/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb', '/home/ubuntu/linux-headers-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb',   '/home/ubuntu/linux-image-4.19.234.mptcp_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-libc-dev_20220311125841-1_amd64.deb',               '/home/ubuntu/linux-libc-dev_20220311125841-1_amd64.deb')
    slice.get_node(name=node_name).upload_file('/tmp/linux-mptcp_v0.95.2_20220311125841-1_all.deb',            '/home/ubuntu/linux-mptcp_v0.95.2_20220311125841-1_all.deb')
    slice.get_node(name=node_name).execute("sudo dpkg -i linux*.deb")
    slice.get_node(name=node_name).execute("sudo apt-get install -f")

Restart the nodes:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo reboot")

and wait for them to come up again:

In [ ]:
slice.wait_ssh(progress=True)

Check that MPTCP kernel is loaded (`4.19.X.mptcp`):

In [ ]:
for node_name in ["client", "server"]:
    output = slice.get_node(name=node_name).execute("uname -r")[0].strip()
    print(f"Kernel on {node_name}: {output}")
    
    if "mptcp" not in output: # halt notebook execution if kernel is not installed!
        raise Exception ("No MPTCP kernel on %s" % node_name)

### Configure network 

Now we can configure IP addresses on every network interface:

In [ ]:
print(f"{slice.list_interfaces()}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

slice.get_node(name='client').ip_addr_add(addr="192.168.10.2", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="client-if_c1-p1"))
slice.get_node(name='client').ip_addr_add(addr="192.168.20.2", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="client-if_c2-p1"))
                                          
slice.get_node(name='server').ip_addr_add(addr="192.168.3.1", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="server-if_s1-p1"))
slice.get_node(name='server').ip_addr_add(addr="192.168.4.1", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="server-if_s2-p1"))

slice.get_node(name='router1').ip_addr_add(addr="192.168.1.2", subnet=IPv4Network("192.168.1.0/24"), interface=slice.get_interface(name="router1-if_r1_l-p1"))
slice.get_node(name='router1').ip_addr_add(addr="192.168.3.2", subnet=IPv4Network("192.168.3.0/24"), interface=slice.get_interface(name="router1-if_r1_r-p1"))
                                          
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.10.1", subnet=IPv4Network("192.168.10.0/24"), interface=slice.get_interface(name="emulator1-if_e1_l-p1"))
slice.get_node(name='emulator1').ip_addr_add(addr="192.168.1.1",  subnet=IPv4Network("192.168.1.0/24"),  interface=slice.get_interface(name="emulator1-if_e1_r-p1"))

slice.get_node(name='router2').ip_addr_add(addr="192.168.2.2", subnet=IPv4Network("192.168.2.0/24"), interface=slice.get_interface(name="router2-if_r2_l-p1"))
slice.get_node(name='router2').ip_addr_add(addr="192.168.4.2", subnet=IPv4Network("192.168.4.0/24"), interface=slice.get_interface(name="router2-if_r2_r-p1"))
                                           
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.20.1", subnet=IPv4Network("192.168.20.0/24"), interface=slice.get_interface(name="emulator2-if_e2_l-p1"))
slice.get_node(name='emulator2').ip_addr_add(addr="192.168.2.1",  subnet=IPv4Network("192.168.2.0/24"),  interface=slice.get_interface(name="emulator2-if_e2_r-p1"))

### Set up multipath networking

#### Client and server

Install some other utility software on client and server:

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute("sudo apt update && sudo apt upgrade -y; sudo apt install -y make; sudo apt install build-essential")
    slice.get_node(name=node_name).execute("sudo apt -y install iperf3 net-tools moreutils; sudo apt-get -y install pkg-config")

Download and install `iproute-mptcp` on client and server:


In [ ]:
!git clone https://github.com/multipath-tcp/iproute-mptcp.git

In [ ]:
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).upload_directory('iproute-mptcp', '/home/ubuntu/')
    slice.get_node(name=node_name).execute("cd /home/ubuntu/iproute-mptcp; sudo make; sudo make install")

Enable multipath kernel modules on client and server:

In [ ]:
cmds = """
# run sysctl commands to enable mptcp
sudo sysctl -w net.mptcp.mptcp_enabled=1 
sudo sysctl -w net.mptcp.mptcp_checksum=0

# load and configure mptcp congestion control algorithms
sudo modprobe mptcp_balia 
sudo modprobe mptcp_coupled 
sudo modprobe mptcp_olia 
sudo modprobe mptcp_wvegas

# configure mptcp congestion control algorithm to one
sudo sysctl -w net.ipv4.tcp_congestion_control=balia
"""
for node_name in ["client", "server"]:
    slice.get_node(name=node_name).execute(cmds)

Bring up interfaces and add routes on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.10.1")
slice.get_node(name="client").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.20.1")

Enable MPTCP in the experiment interfaces on client:

In [ ]:
for iface in slice.get_node(name="client").get_interfaces():
    slice.get_node(name="client").execute("sudo ip link set dev " + iface.get_os_interface() + " multipath on")

and disable on the control interface:

(note: currently broken, see https://learn.fabric-testbed.net/forums/topic/error-on-get_management_os_interface/
but it doesn't seem to matter)

In [ ]:
control_iface = slice.get_node(name="client").get_management_os_interface()
slice.get_node(name="client").execute("sudo ip link set dev " + control_iface + " multipath off")

Now, add MP routes on client:

In [ ]:
cmds = """
iface1=$(ifconfig | grep -B1 'inet 192.168.10.2' | head -n1 | cut -f1 -d:)
iface2=$(ifconfig | grep -B1 'inet 192.168.20.2' | head -n1 | cut -f1 -d:)

sudo ip rule add from 192.168.10.2 table 1 
sudo ip rule add from 192.168.20.2 table 2 
sudo ip route add 192.168.10.0/24 dev $iface1 scope link table 1
sudo ip route add 192.168.20.0/24 dev $iface2 scope link table 2 
sudo ip route add 192.168.3.0/24 via 192.168.10.1 dev $iface1 table 1 
sudo ip route add 192.168.4.0/24 via 192.168.20.1 dev $iface2 table 2
"""
slice.get_node(name="client").execute(cmds)

Check configuration on client:

In [ ]:
client_ip_ad = slice.get_node(name="client").execute('ip addr')
client_ip_rt = slice.get_node(name="client").execute('ip route')
client_ip_r1 = slice.get_node(name="client").execute('ip route show table 1')
client_ip_r2 = slice.get_node(name="client").execute('ip route show table 2')
for output in [client_ip_ad, client_ip_rt, client_ip_r1, client_ip_r2]:
    print(*output, sep='\n')

Bring up interfaces and add routes on server:

In [ ]:
for iface in slice.get_node(name="server").get_interfaces():
    iface.ip_link_up()
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.3.2")
slice.get_node(name="server").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.4.2")

Check configuration on server:

In [ ]:
server_ip_ad = slice.get_node(name="server").execute('ip addr')
server_ip_rt = slice.get_node(name="server").execute('ip route')
print(*server_ip_ad, sep='\n')
print(*server_ip_rt, sep='\n')

#### Gateways

Bring up links:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    for iface in slice.get_node(name=node_name).get_interfaces():
        iface.ip_link_up()

Enable IP forwarding:

In [ ]:
for node_name in ["emulator1", "emulator2", "router1", "router2"]:
    slice.get_node(name=node_name).execute("sudo sysctl -w net.ipv4.ip_forward=1")

Add some routes throughout the network:

In [ ]:
slice.get_node(name="emulator1").ip_route_add(subnet=IPv4Network("192.168.3.0/24"), gateway="192.168.1.2")
slice.get_node(name="emulator2").ip_route_add(subnet=IPv4Network("192.168.4.0/24"), gateway="192.168.2.2")
slice.get_node(name="router1").ip_route_add(subnet=IPv4Network("192.168.10.0/24"), gateway="192.168.1.1")
slice.get_node(name="router2").ip_route_add(subnet=IPv4Network("192.168.20.0/24"), gateway="192.168.2.1")

Add delay at emulator nodes on the "right" side of the network:

In [ ]:
em_iface_r = slice.get_node(name="emulator1").get_interface(name="emulator1-if_e1_r-p1").get_os_interface()
slice.get_node(name="emulator1").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')
em_iface_r = slice.get_node(name="emulator2").get_interface(name="emulator2-if_e2_r-p1").get_os_interface()
slice.get_node(name="emulator2").execute('sudo tc qdisc replace dev ' + em_iface_r + ' root netem delay 30ms limit 60000')

Add capacity limit on the router nodes on the "right" side of the network"

In [ ]:
cmds = """
sudo tc qdisc del dev IFACE root # don’t worry if you get RTNETLINK  error
sudo tc qdisc replace dev IFACE root handle 1: htb default 3 
sudo tc class add dev IFACE parent 1: classid 1:3 htb rate 100mbit 
sudo tc qdisc add dev IFACE parent 1:3 handle 3: bfifo limit 375000 
"""

rt_iface_r = slice.get_node(name="router1").get_interface(name="router1-if_r1_r-p1").get_os_interface()
slice.get_node(name="router1").execute(cmds.replace("IFACE", rt_iface_r))
rt_iface_r = slice.get_node(name="router2").get_interface(name="router2-if_r2_r-p1").get_os_interface()
slice.get_node(name="router2").execute(cmds.replace("IFACE", rt_iface_r))

View all the routes:

In [ ]:
for node_name in ["client", "server", "emulator1", "emulator2", "router1", "router2"]: 
    ip_rt = slice.get_node(name=node_name).execute('ip route')
    print("Routes at " + node_name + ":")
    print(*ip_rt, sep='\n')

## Validate the multipath configuration

Start an iperf3 server on the server node (in daemon mode -D option, so this command won't block).
Then start an iperf3 client on the client node.

In [ ]:
slice.get_node(name="server").execute("iperf3 -s -1 -D")

In [ ]:
iperf_out = slice.get_node(name="client").execute("iperf3 -f m -c 192.168.3.1 -C 'balia' -P 1 -i 1 -t 10")
#iperf_out = slice.get_node(name="client").execute("iperf3 -c 192.168.3.1 -C 'balia' -i 10")
print(*iperf_out, sep='\n')

Confirm that you get about 200 Mbps throughput, then you can go ahead and run the experiment to replay the wireless link traces in the next section.

## "Play back" link traces on the multipath configuration

### Set up experiment

Select Congestion Control algorithm in client and server

In [42]:
ccAlgo="balia"
slice.get_node(name="client").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="server").execute("sudo sysctl -w net.mptcp.mptcp_enabled=1")
slice.get_node(name="client").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)
slice.get_node(name="server").execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + ccAlgo)

net.mptcp.mptcp_enabled = 1
net.mptcp.mptcp_enabled = 1
net.ipv4.tcp_congestion_control = balia
net.ipv4.tcp_congestion_control = balia


('net.ipv4.tcp_congestion_control = balia\n', '')

Copy the scripts and Trace files to router1 and router2

In [43]:
MPTracesRootFolder="/home/fabric" + "/work/Multipath-Wireless-Link-Traces"
nodeHomeFolder="/home/ubuntu"
print(MPTracesRootFolder)
print(nodeHomeFolder)

/home/fabric/work/Multipath-Wireless-Link-Traces
/home/ubuntu


In [ ]:
for node_name in ["router1", "router2"]:
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/FABRIC", nodeHomeFolder)
    slice.get_node(name=node_name).upload_directory(MPTracesRootFolder + "/Traces", nodeHomeFolder)    

### Execute experiment

In [51]:
print("Starting the Bulk Transfer Reference experiment...") 
# select path and trial
path="8" 
trial="1"

# get the trace pair file names to feed into the routers
wifiTraceFile=nodeHomeFolder + "/Traces/" + path + "_" + trial + "_wifi.csv"  
cellularTraceFile=nodeHomeFolder + "/Traces/"+ path + "_" + trial + "_cellular.csv" 
print("\twith trace files: " + cellularTraceFile + "  " + wifiTraceFile + "  ")

outputDir="rawOutputs"
outputFileName=nodeHomeFolder + "/" + outputDir + "/" + "rawResults" + "_path" + path + "_trial" + trial + ".txt"
print("\toutput file at the client node: " + outputFileName)


Starting the Bulk Transfer Reference experiment...
	with trace files: /home/ubuntu/Traces/8_1_cellular.csv  /home/ubuntu/Traces/8_1_wifi.csv  
	output file at the client node: /home/ubuntu/rawOutputs/rawResults_path8_trial1.txt


In [53]:
# start scripts in router1 and router2 (in non-blocking way using *at* command) 
# so the bandwidths at the routers are regulated based on the collected
# trace pairs (wifi trace on router 1 and cellular trace in router2, respectively)
# important note: make sure to run the script with /bin/bash to make it work with at command
router1Cmd="echo /bin/bash " + nodeHomeFolder + "/FABRIC/tputVary.sh " + wifiTraceFile + " | at now"
router2Cmd="echo /bin/bash " + nodeHomeFolder + "/FABRIC/tputVary.sh "+ cellularTraceFile +" | at now"

print("\texecuting router commands: " + router1Cmd + " " + router2Cmd)
slice.get_node(name="router1").execute(router1Cmd)
slice.get_node(name="router2").execute(router2Cmd)     

	executing router commands: echo /bin/bash /home/ubuntu/FABRIC/tputVary.sh /home/ubuntu/Traces/8_1_wifi.csv | at now echo /bin/bash /home/ubuntu/FABRIC/tputVary.sh /home/ubuntu/Traces/8_1_cellular.csv | at now
job 8 at Thu Apr 20 15:59:00 2023
job 6 at Thu Apr 20 15:59:00 2023
 

('',
 'warning: commands will be executed using /bin/sh\njob 6 at Thu Apr 20 15:59:00 2023\n')

In [54]:
print("\trunning the iperf at the server...")
slice.get_node(name="server").execute('iperf3 -s -i 0.1 -D -1') # run as daemon and stop after 1 client connection

	running the iperf at the server...


('', '')

In [55]:
print("\tcreate output directory " + outputDir + " in the client to store iperf output...") 
slice.get_node(name="client").execute("mkdir -p " + nodeHomeFolder + "/" + outputDir )

	create output directory rawOutputs in the client to store iperf output...


('', '')

In [56]:
clientCmd="iperf3 -f m -c 192.168.3.1 -C '" + ccAlgo + "' -P 1 -i 1 -t 10" + " > " + outputFileName 
#clientCmd="iperf3 -f m -c 192.168.3.1 -C '" + ccAlgo + "' -P 1 -i 1 -t 10"  
print("\tclient iperf command: " + clientCmd)
print("\trunning iperf at the client...") 
slice.get_node(name="client").execute(clientCmd)
#iperf_out = slice.get_node(name="client").execute(clientCmd)
#print(*iperf_out, sep='\n')  

	client iperf command: iperf3 -f m -c 192.168.3.1 -C 'balia' -P 1 -i 1 -t 10 > /home/ubuntu/rawOutputs/rawResults_path8_trial1.txt
	running iperf at the client...


('', '')

In [57]:
print("\tkilling the trace pair feeding scripts at both the routers to end this experiment...")        
killCmd="kill -9 $(ps -ef | grep -i tput | awk '{print $2}')"
slice.get_node(name="router1").execute(killCmd)
slice.get_node(name="router2").execute(killCmd)       
        
print("Finished the Bulk Transfer Reference experiment.")  

	killing the trace pair feeding scripts at both the routers to end this experiment...
Finished the Bulk Transfer Reference experiment.


### Retrieve data from experiment

Downloading the raw results (output) file from client node to the FABRIC environment.


In [58]:
slice.get_node(name="client").download_directory('/home/fabric/work/' ,  outputDir )
print("Experiment results are downloaded to this folder in this notebook: " + outputDir)

Experiment results are downloaded to this folder in this notebook: rawOutputs


## Delete slice when finished

In [ ]:
#fablib.delete_slice(slice_name)